In [5]:
import os
from PIL import Image
import imagehash
import cv2
import numpy as np


In [ ]:

def rename_and_format(source_directory, destination_directory_train, destination_directory_valid, destination_format='JPEG'):
    files = os.listdir(source_directory)
    size_dir = len(files)
    counter = 1

    for file in files:
        source_path = os.path.join(source_directory, file)

        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):

            if counter / size_dir >  0.8:
                destination_directory = destination_directory_valid 
            else:
                destination_directory = destination_directory_train

            image = Image.open(source_path)

            new_name = f'image_{counter:04d}.{destination_format.lower()}'
            counter += 1

            destination_path = os.path.join(destination_directory, new_name)

            image.save(destination_path, destination_format)

    print("Done!")

rename_and_format("data/fire/original_photos", "data/fire/train","data/fire/valid" )



In [4]:

def find_and_delete_duplicate_images(directory):
    hashes = {}

    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)

            with Image.open(file_path) as img:
                h = imagehash.phash(img)

            if h in hashes:
                print(f'Duplicate image found: {file_path} and {hashes[h]}')

                os.remove(file_path)

                print(f'Duplicate image deleted: {file_path}')
            else:
                hashes[h] = file_path

directories_to_check = ["data/fire/train", "data/fire/valid"]

for directory in directories_to_check:
    find_and_delete_duplicate_images(directory)
    


In [7]:


def augment_data(directory):

    source_directory = directory
    destination_directory = directory

    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    files = os.listdir(source_directory)

    for file in files:
        source_path = os.path.join(source_directory, file)
        destination_path = os.path.join(destination_directory, file)

        image = cv2.imread(source_path)

        # Horizontal flipping
        flipped_image = cv2.flip(image, 1)
        cv2.imwrite(os.path.join(destination_directory, f"flipped_{file}"), flipped_image)

        # Rotation at different angles (e.g., 30 degrees)
        rotation_angles = [30, -30]
        for angle in rotation_angles:
            rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
            rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
            cv2.imwrite(os.path.join(destination_directory, f"rotated_{angle}_{file}"), rotated_image)

        # Zoom in/out
        zoom_factors = [1.2, 0.8]
        for zoom_factor in zoom_factors:
            zoomed_image = cv2.resize(image, None, fx=zoom_factor, fy=zoom_factor)
            cv2.imwrite(os.path.join(destination_directory, f"zoomed_{zoom_factor}_{file}"), zoomed_image)

        # Vertical/horizontal shift
        shifts = [(50, 0), (0, 50)]
        for shift in shifts:
            shift_matrix = np.float32([[1, 0, shift[0]], [0, 1, shift[1]]])
            shifted_image = cv2.warpAffine(image, shift_matrix, (image.shape[1], image.shape[0]))
            cv2.imwrite(os.path.join(destination_directory, f"shifted_{shift[0]}_{shift[1]}_{file}"), shifted_image)

        # Brightness/contrast adjustment
        alpha = 1.5  # Contrast factor
        beta = 30    # Brightness value added
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        cv2.imwrite(os.path.join(destination_directory, f"adjusted_{alpha}_{beta}_{file}"), adjusted_image)


for directory in directories_to_check:
    augment_data(directory)


In [ ]:
print(len(os.listdir("data/fire/train")),len(os.listdir("data/fire/valid")))